In [9]:
from __future__ import absolute_import, division

from keras.preprocessing.sequence import pad_sequences

import Model
import numpy as np 

import senteval
import pickle


PATH_SENTEVAL = '../'
PATH_TO_DATA = 'data'

maxLength = 30    
wordDim = 300    # 300 
num_neurons = [2*wordDim,2*wordDim]   # [600,600f]
extractLayer = ['X_input_second','X_input_mask_2']
kernel_reg = 0.001
batch_size = 400
merge_mode = 'concat'

# _weights-epoch-01-loss-7.0428-val_loss-8.5336.hdf5,  _weights-epoch-01-loss-9.0715-val_loss-8.6046.hdf5
checkpoint = "../checkpoint_mask_2500_MSE/loss-0.1413-cata_acc-0.7126-val_loss-0.1590-cate_acc_val-0.6665.hdf5"  # Change 
processedPath = "../processed/"
tokenizerFile = 'tokenizer.txt'
embeddingFile =  'embedding_matrix.txt'
embeddingSize = 678350

with open(processedPath + tokenizerFile , "rb") as input_file:
    t = pickle.load(input_file)

with open(processedPath + embeddingFile , "rb") as input_file:
    embedding_matrix = pickle.load(input_file) 
    
vocab_size = len(embedding_matrix)

count = 0

# SentEval prepare and batcher
def prepare(params, samples):
    return

def batcher(params, batch):
    global count
    global t
    global embedding_matrix
    global model 
    sentences = [' '.join(s) for s in batch]
    
    # Preprocessing
    sentences = Model.preprocessing(sentences)
    sentences = t.texts_to_sequences(sentences)
    sentences = pad_sequences(sentences, maxlen=maxLength, padding='post')
    # Restore model 
    
    if count == 0:
        print("build model")
        model = Model.buildModel(sentences,kernel_reg,num_neurons,merge_mode, embeddingSize,maxLength, wordDim,np.zeros([embeddingSize,wordDim]),False)
        model.load_weights(checkpoint)
        count += 1

    # Extract th output 
    output = Model.extractHiddenState(extractLayer,model,sentences)
    output = np.concatenate((output[0],output[1]),axis=1)
    
    return output



In [10]:
tasks = ['MR', 'SUBJ','CR','MPQA','TREC','MRPC']
result = []
for task in tasks:
    print(task)
    params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 10}
    params_senteval['classifier'] = {'nhid': 0, 'optim': 'adam', 'batch_size': 512,
                                     'tenacity': 5, 'epoch_size': 5}
    # Set up logger
    se = senteval.engine.SE(params_senteval, batcher, prepare)
    transfer_tasks =    [task]  # ['CR','MPQA','TREC','MRPC']
    # 'MR', 'CR' ,'MPQA', 'SUBJ','TREC', 'MRPC','SICKEntailment', 'SICKRelatedness'  ,'TREC','MRPC'
    results = se.eval(transfer_tasks)
    result.append(results)
    
# params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 10}
# params_senteval['classifier'] = {'nhid': 0, 'optim': 'adam', 'batch_size': 512,
#                                  'tenacity': 5, 'epoch_size': 10}
# # Set up logger
# se = senteval.engine.SE(params_senteval, batcher, prepare)
# transfer_tasks = ['MR', 'SUBJ','CR','MPQA','TREC','MRPC']

# # 'MR', 'CR' ,'MPQA', 'SUBJ','TREC', 'MRPC','SICKEntailment', 'SICKRelatedness'
# results = se.eval(transfer_tasks)
# print(results)


MR
Pre processing Data
build model
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre processing Data
Pre p

In [11]:
result

[{'MR': {'devacc': 73.69, 'acc': 72.94, 'ndev': 10662, 'ntest': 10662}},
 {'SUBJ': {'devacc': 90.1, 'acc': 89.73, 'ndev': 10000, 'ntest': 10000}},
 {'CR': {'devacc': 77.86, 'acc': 76.45, 'ndev': 3775, 'ntest': 3775}},
 {'MPQA': {'devacc': 84.98, 'acc': 84.6, 'ndev': 10606, 'ntest': 10606}},
 {'TREC': {'devacc': 77.57, 'acc': 80.4, 'ndev': 5452, 'ntest': 500}},
 {'MRPC': {'devacc': 73.92,
   'acc': 73.45,
   'f1': 81.58,
   'ndev': 4076,
   'ntest': 1725}}]